# Car Fire Detection

Using the pre-trained DenseNet and the added layer, learn images related to car fires.

For learning, 1,000 images of accident and 1,000 images of non-accident were used, so total 2,000 images were used.

See the comments for details.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import datasets
import IPython.display as display
from PIL import Image
from glob import glob
import pathlib

train_dir = pathlib.Path('./images/train')   # load train images
validation_dir = pathlib.Path('./images/validation')  # load validation images
train_image_count = len(list(train_dir.glob('*/*.jpg')))
validation_image_count = len(list(validation_dir.glob('*/*.jpg')))
print(train_image_count)
print(validation_image_count)

CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')])
CLASS_NAMES

accidents = list(train_dir.glob('accident/*'))

for image_path in accidents[:3]:
    display.display(Image.open(str(image_path)))

# apply horizontal flip for data augmentation
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True) 
validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True) 

BATCH_SIZE = 32   # batch size is 32
IMG_HEIGHT = 160
IMG_WIDTH = 160
STEPS_PER_EPOCH = np.ceil(train_image_count/BATCH_SIZE)  # epoch for each learning step 
VALIDATION_STEPS = np.ceil(validation_image_count/BATCH_SIZE)
print(STEPS_PER_EPOCH)
print(VALIDATION_STEPS)

# loading images from directory 
train_data_gen = image_generator.flow_from_directory(directory='./images/train', 
                                                     batch_size=BATCH_SIZE,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(directory='./images/validation', 
                                                              batch_size=BATCH_SIZE,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')
                        
        
def create_model():
    IMG_SHAPE = (160,160,3)   # input images are 160 x 160 x 3 (W x H x C)

    base_model = tf.keras.applications.DenseNet201(   # using pretrained model : DenseNet201
        include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE,
        pooling='avg', classes=1000
    )

    base_model.trainable = True    # base model should also be learned

    model = tf.keras.Sequential([   # add some layers
    base_model,
    tf.keras.layers.Dense(1920, activation='relu'),  
    tf.keras.layers.Dropout(0.2),  # in order for avoiding overfitting, apply dropout
    tf.keras.layers.Dense(1)  # output layer must be 1 since result will be binary (0 : accident, 1 : non accident)
    ])

    base_learning_rate = 0.0001  # learning rate is 0.0001 (it's a empirically derived value)
    model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),   # use optimizer with Adam
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),   # loss function is binary cross entropy
    metrics=['accuracy']
    )
    return model

model = create_model()
model.summary()
len(model.trainable_variables)

# training model 
model.fit(train_data_gen, batch_size=BATCH_SIZE, validation_data=val_data_gen, steps_per_epoch=STEPS_PER_EPOCH, validation_steps=VALIDATION_STEPS)

# save model in a specific directory
model.save('./model', save_format='tf')